In [22]:
import pandas as pd
import openai
import re
from sentence_transformers import SentenceTransformer
import torch
import sys

print(sys.executable)


/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9


In [2]:
df = pd.read_csv('narc.csv')
moas = df['MoA']

In [21]:
!sys.executable -m pip install sentence-transformers

  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 7.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.4 MB/s eta 0:00:00:00:0100:01
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 4.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 5.2 MB/s eta 0:00:00
  Using cached PyYAML-6.0-cp39-cp39-macosx_10_9_x86_64.whl (197 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 8.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.4/294.4 kB 7.8 MB/s eta 0:00:0000:01
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 MB 7.5 MB/s eta 0:00:0000:0100:01
  Using cached sympy-1.11.1-

In [3]:
def parse(answer):
    question = "given some examples, identify the receptor type and binding mode that come before the mechanism of action. answer n/a if not specified."
    prompt = "Example: the receptors are: histamine h3. the subtypes are: n/a. the binding modes are: inverse agonist. therefore the mechanism of action is: selective histamine h3 receptor inverse agonist. additionally, modafinil is a wakefulnesspromoting agent.\n"
    prompt += "Answer: histamine h3 receptor inverse agonist\n"
    prompt += "Example: the receptors are: dopamine. the subtypes are: not specified in the abstract. the binding modes are: agonist. therefore the mechanism of action is: dopamine agonist.\n"
    prompt += "Answer: dopamine agonist"
    prompt += "the mechanism of action is: putative central alpha1 agonist. however, the article does not mention any specific receptor subtypes that the drug modafinil is being evaluated with.\n"
    prompt += "Answer: putative cnetral alpha1 agonist"
    prompt += "Example: {}\n".format(answer)
    prompt += "Answer: "
    
    try:
        response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                {"role": "system", "content": "You are a very helpful medical researcher."},
                {"role": "user", "content": prompt}]
        )
        text = response['choices'][0]['message']['content']
        print(text)
    except:
        print("Error")
        text = "None"
    return text
        
def get_MoA(answer):
    answer = answer.lower()
    answer = answer.replace("(", "")
    answer = answer.replace(")", "")
    answer = answer.replace("-", "")
    return parse(answer)
    
    


In [5]:
cook_moa = []
for moa in moas:
    cook_moa += [parse(moa)]


selective histamine H3 receptor inverse agonist.
No example given.
histamine-3 receptor inverse agonist.
histamine 3 (H3) receptor
histamine H(3) receptor inverse agonist
H3 antagonist/inverse agonist.
GABA(B) agonist.
mechanism of action not specified
selective histamine 3 receptor antagonist/inverse agonist
histamine H3 receptor inverse agonist, specifically pitolisant.
putative central alpha-1 agonist.
GABA agonist (specifically GABA-B receptor agonist and/or GHB receptor agonist).
GHB-/GABA-B receptor agonist
GABA agonist (specific receptor subtype not specified in article)
dopamine D2 receptor agonist.
GHB/gamma-aminobutyric acid B receptor agonist
N/A
selective histamine H3 receptor inverse agonist
GABAŒ≤ receptor agonist.
GHB-/GABA-B receptor agonist.
dopamine agonist
No mechanism of action/receptor subtype information provided in the article.
GABA-B receptor agonists/pharmacology.
dual orexin receptor antagonist
histamine H3 receptor therapeutic use.
serotonin 5HT2 antagonist
d

interaction with dopamine neurotransmitter system, no specific receptor subtype mentioned.
No drugs or mechanisms of action mentioned in the article.
not specified or unknown receptor subtype, with possible noradrenergic stimulation of the prefrontal cortex for working memory effects.
No information provided on mechanism of action or receptor subtype(s) being evaluated.
central nervous system stimulant. No specific receptor subtype is mentioned.
Not specified
central nervous system stimulant
The article does not provide information on receptor subtype or mechanism of action for modafinil and methylphenidate.
central nervous system stimulant (mechanism of action not related to specific receptors)
Central nervous system stimulant.
nonamphetamine Central Nervous System stimulant with unknown receptor subtype(s) being targeted.
The mechanism of action is wake-promoting, but there is no specific target receptor or subtype mentioned in the article.
not specified
melatonin receptor agonist.
e

In [4]:
openai.api_key = "" # vedant key 

In [6]:
print(cook_moa)

['selective histamine H3 receptor inverse agonist.', 'No example given.', 'histamine-3 receptor inverse agonist.', 'histamine 3 (H3) receptor', 'histamine H(3) receptor inverse agonist', 'H3 antagonist/inverse agonist.', 'GABA(B) agonist.', 'mechanism of action not specified', 'selective histamine 3 receptor antagonist/inverse agonist', 'histamine H3 receptor inverse agonist, specifically pitolisant.', 'putative central alpha-1 agonist.', 'GABA agonist (specifically GABA-B receptor agonist and/or GHB receptor agonist).', 'GHB-/GABA-B receptor agonist', 'GABA agonist (specific receptor subtype not specified in article)', 'dopamine D2 receptor agonist.', 'GHB/gamma-aminobutyric acid B receptor agonist', 'N/A', 'selective histamine H3 receptor inverse agonist', 'GABAŒ≤ receptor agonist.', 'GHB-/GABA-B receptor agonist.', 'dopamine agonist', 'No mechanism of action/receptor subtype information provided in the article.', 'GABA-B receptor agonists/pharmacology.', 'dual orexin receptor antago

In [23]:


model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

sentence1 = ['histamine H5 receptor antagonist']
sentence2 = ['serotonin-5-HT1A receptor partial agonist']
sentence3 = ['histamine H5 receptor']

embedding1 = torch.from_numpy(model.encode(sentence1))
embedding2 = torch.from_numpy(model.encode(sentence2))
embedding3 = torch.from_numpy(model.encode(sentence3))

print(embedding1, embedding2)

cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

print(cos(embedding1, embedding2), cos(embedding2, embedding3), cos(embedding1, embedding3))



tensor([[-1.2869e-01, -3.1490e-01, -1.8536e-01, -3.6028e-01,  6.9758e-02,
         -6.0819e-02,  4.4758e-01,  2.4126e-01, -4.6531e-01, -3.7491e-01,
         -6.9673e-02, -2.2717e-01, -2.4573e-01,  5.3794e-01,  3.0839e-03,
         -4.5915e-01,  4.8488e-01,  1.8270e-01,  2.0694e-02, -3.1753e-02,
         -3.8422e-01,  8.6955e-01, -3.7452e-02, -3.6183e-01, -2.0668e-01,
         -2.5598e-01,  5.0824e-01, -3.8904e-01, -5.8430e-01, -1.5492e-01,
          2.3613e-01,  6.0958e-02,  4.5978e-02, -6.3792e-01, -2.3208e-01,
         -4.3157e-01, -2.8791e-01, -8.1075e-02, -7.9761e-02, -3.8459e-01,
         -2.9928e-01,  5.4298e-01, -2.0128e-01,  4.4757e-01, -4.1070e-03,
          1.4170e-01, -2.5504e-01, -3.6366e-01,  2.8574e-01,  7.6042e-01,
          4.4506e-03, -3.6205e-01,  3.3727e-02,  2.6197e-01,  6.6345e-02,
         -1.5012e-01, -6.9741e-01,  4.4046e-01, -5.2889e-01,  4.6590e-01,
         -5.9348e-01,  4.6951e-01,  2.6011e-01, -1.3229e-02,  6.5644e-01,
         -4.6586e-02, -1.5680e-01, -6.

In [26]:
def get_similarity(a, b):
    embedding_a = torch.from_numpy(model.encode([a]))
    embedding_b = torch.from_numpy(model.encode([b]))
    return cos(embedding_a, embedding_b)

def cluster_stuff(MoA_list):
    clusters = {} 
    threshold = 0.85 # have to hyerparameter tune this
    for ele in MoA_list:
        print("hi")
        best = 0.0
        cluster_to_match = ""
        for compare_cluster in clusters:
            if get_similarity(compare_cluster, ele) > best:
                best = get_similarity(compare_cluster, ele)
                cluster_to_match = compare_cluster
        if best > threshold:
            clusters[cluster_to_match].append(ele)
        else:
            clusters[ele] = [ele]
    print(clusters)


In [27]:
cluster_stuff(cook_moa)

{'selective histamine H3 receptor inverse agonist.': ['selective histamine H3 receptor inverse agonist.', 'histamine-3 receptor inverse agonist.', 'histamine 3 (H3) receptor', 'histamine H(3) receptor inverse agonist', 'selective histamine 3 receptor antagonist/inverse agonist', 'histamine H3 receptor inverse agonist, specifically pitolisant.', 'selective histamine H3 receptor inverse agonist', 'histamine H3 receptor therapeutic use.'], 'No example given.': ['No example given.'], 'H3 antagonist/inverse agonist.': ['H3 antagonist/inverse agonist.'], 'GABA(B) agonist.': ['GABA(B) agonist.'], 'mechanism of action not specified': ['mechanism of action not specified', 'unknown/mechanism of action not specified', 'unknown/mechanism of action not specified.', 'Mechanism of action not specified.', 'unknown/mechanism of action not specified'], 'putative central alpha-1 agonist.': ['putative central alpha-1 agonist.'], 'GABA agonist (specifically GABA-B receptor agonist and/or GHB receptor agoni

In [ ]:
for c